In [ ]:
import fastf1
import pandas as pd

def get_driver_data(driver)
    session = fastf1.get_session(2024, 'Silverstone', 'R')
    session.load()
    driver_data = session.laps.pick_driver(driver)
    driver_df = pd.DataFrame(driver_data)
    return driver_df
    

In [52]:
#seconds for laptime, and drop previous column
def adjust_laptimes(dataframe):
    dataframe["Laptime"] = dataframe["LapTime"].astype(int) * 1e-9
    dataframe["Position(s)"] = dataframe["Position"].astype(int)
    new_df = dataframe.drop(columns = ["LapTime", "Position"])
    new_df.rename({"Positions(s)":"Position"})
    return new_df

In [53]:
def remove_extras(dataframe):
    new_df = dataframe.drop(columns = ["Time", 'DriverNumber', 'Team', 'LapStartTime', 'LapStartDate', 'Deleted', "DeletedReason" 'FastF1Generated', 'IsAccurate'])
    return new_df

,Time,Driver,DriverNumber,LapNumber,Stint,PitOutTime,PitInTime,Sector1Time,Sector2Time,Sector3Time,...,Team,LapStartTime,LapStartDate,TrackStatus,Position,Deleted,DeletedReason,FastF1Generated,IsAccurate,Laptime
726,0 days 01:00:00.089000,BOT,77,1.0,1.0,NaT,NaT,NaT,0 days 00:00:40.380000,0 days 00:00:26.634000,...,Kick Sauber,0 days 00:58:15.512000,2024-07-07 14:03:12.541,1,19.0,False,,False,False,104.290
727,0 days 01:01:35.457000,BOT,77,2.0,1.0,NaT,NaT,0 days 00:00:30.060000,0 days 00:00:39.194000,0 days 00:00:26.114000,...,Kick Sauber,0 days 01:00:00.089000,2024-07-07 14:04:57.118,1,18.0,False,,False,True,95.368
728,0 days 01:03:09.904000,BOT,77,3.0,1.0,NaT,NaT,0 days 00:00:30.096000,0 days 00:00:38.414000,0 days 00:00:25.937000,...,Kick Sauber,0 days 01:01:35.457000,2024-07-07 14:06:32.486,1,18.0,False,,False,True,94.447
729,0 days 01:04:44.226000,BOT,77,4.0,1.0,NaT,NaT,0 days 00:00:29.890000,0 days 00:00:38.839000,0 days 00:00:25.593000,...,Kick Sauber,0 days 01:03:09.904000,2024-07-07 14:08:06.933,1,19.0,False,,False,True,94.322
730,0 days 01:06:17.994000,BOT,77,5.0,1.0,NaT,NaT,0 days 00:00:29.920000,0 days 00:00:38.115000,0 days 00:00:25.733000,...,Kick Sauber,0 days 01:04:44.226000,2024-07-07 14:09:41.255,1,19.0,False,,False,True,93.768
731,0 days 01:07:51.454000,BOT,77,6.0,1.0,NaT,NaT,0 days 00:00:29.755000,0 days 00:00:37.957000,0 days 00:00:25.748000,...,Kick Sauber,0 days 01:06:17.994000,2024-07-07 14:11:15.023,1,19.0,False,,False,True,93.460
732,0 days 01:09:25.102000,BOT,77,7.0,1.0,NaT,NaT,0 days 00:00:29.919000,0 days 00:00:37.898000,0 days 00:00:25.831000,...,Kick Sauber,0 days 01:07:51.454000,2024-07-07 14:12:48.483,1,19.0,False,,False,True,93.648
733,0 days 01:10:58.663000,BOT,77,8.0,1.0,NaT,NaT,0 days 00:00:29.899000,0 days 00:00:37.968000,0 days 00:00:25.694000,...,Kick Sauber,0 days 01:09:25.102000,2024-07-07 14:14:22.131,1,19.0,False,,False,True,93.561
734,0 days 01:12:32.308000,BOT,77,9.0,1.0,NaT,NaT,0 days 00:00:29.917000,0 days 00:00:37.913000,0 days 00:00:25.815000,...,Kick Sauber,0 days 01:10:58.663000,2024-07-07 14:15:55.692,1,19.0,False,,False,True,93.645
735,0 days 01:14:06.095000,BOT,77,10.0,1.0,NaT,NaT,0 days 00:00:29.850000,0 days 00:00:38.666000,0 days 00:00:25.271000,...,Kick Sauber,0 days 01:12:32.308000,2024-07-07 14:17:29.337,1,18.0,False,,False,True,93.787
